In [1]:
import tensorflow as tf
from tensorflow import keras
from component.respiratory_tract import get_respiratory_tract_model
from utils.feature_extraction import  get_MFCCS_v2, oversample
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import glob
import tqdm
import os

In [2]:
def make_data(folder, n_mfcc, num_segment, segment_len, use_oversample=False):
    X1 = glob.glob(os.path.join(folder, 'p', '*.wav'))
    X0 = glob.glob(os.path.join(folder, 'n', '*.wav'))

    y = []
    X = []
    for file in tqdm.tqdm(X1):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(1)
        except:
            pass
    for file in tqdm.tqdm(X0):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(0)
        except:
            pass
    X = np.array(X)
    y = np.array(y)
    if use_oversample:
        X, y = oversample(X, y)
    return X, y

In [9]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = np.arange(len(X))

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = np.random.choice(self.indexes, self.batch_size, replace=False)
        return self.X[indexes], self.y[indexes]

In [10]:
def get_model(n_mfcc, num_segment, weight_path=None):
    model = get_respiratory_tract_model(input_shape=(n_mfcc*3+3, num_segment, 1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])
    if weight_path is not None:
        model.load_weights(weight_path)
    return model

def train_and_test(model_name, train_folder, test_folder, n_mfcc, num_segment, segment_len, clear_cache=True):
    if clear_cache:
        get_MFCCS_v2.cache_clear()
        cached_data = glob.glob("cache/*.npy")
        for file in cached_data:
            os.remove(file)

    #build data
    X, y = make_data(train_folder, n_mfcc, num_segment, segment_len, use_oversample=True)
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weights = {1 - i: x for i,x in enumerate(class_weights)}
    print(f"Class weights: {class_weights}")
    train_generator = DataGenerator(X_train, y_train, 8)
    val_generator = DataGenerator(X_val, y_val, 8)

    #train
    if not os.path.exists(f'logs/{model_name}'):
        os.mkdir(f'logs/{model_name}')
    model = get_model(n_mfcc, num_segment)
    history = model.fit(train_generator, steps_per_epoch=20, validation_steps=50, epochs=1000, validation_data=val_generator, class_weight=class_weights,
                        verbose=1, callbacks=[tf.keras.callbacks.ModelCheckpoint(f'pretrained_weight/{model_name}.h5', monitor="val_loss", save_weights_only=True, verbose=1, mode='min', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir=f'logs/{model_name}')])

    #test
    X_test, y_test = make_data(test_folder,
                     n_mfcc, num_segment, segment_len)
    model.load_weights(f"pretrained_weight/{model_name}.h5")
    y_pred = model.predict(X_test)
    y_pred_r = np.rint(y_pred)
    return history, classification_report(y_pred_r, y_test), roc_auc_score(y_test, y_pred)

In [ ]:
# best param: 39 - 50 - 1024
n_mfccs = 13*np.array([2, 3, 4, 5], dtype=int) #M
num_segments = 10*np.array([3, 5, 7, 10, 12], dtype=int) #S
segment_length = np.exp2([8, 9, 10, 11, 12]).astype(int) #F
train_folder = 'coswara_wavs/train'
test_folder = 'coswara_wavs/test'
result = {}
for _ in range(15):
    while True:
        n_mfcc = random.choice(n_mfccs)
        num_segment = random.choice(num_segments)
        segment_len = random.choice(segment_length)
        if (n_mfcc, num_segment, segment_len) not in result:
            break
    print(f"n_mfcc={n_mfcc}, num_segment={num_segment}, segment_len={segment_len}")
    model_name = f'model_paper_{n_mfcc}_{num_segment}_{segment_len}'
    try:
        history, report, auc_score = train_and_test(model_name, train_folder, test_folder, n_mfcc, num_segment, segment_len)
    except Exception as e:
        result[(n_mfcc, num_segment, segment_len)] = f'error -> {type(e).__name__}: {str(e)}'
    else:
        result[(n_mfcc, num_segment, segment_len)] = history, report, auc_score



  0%|          | 0/296 [00:00<?, ?it/s]

n_mfcc=52, num_segment=50, segment_len=256


 76%|███████▌  | 224/296 [00:41<00:10,  6.98it/s]

cant remove silence, path=coswara_wavs/train\p\s1x2Ze9iVZNf6oFZDAZ4S9x3Cky2_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


 83%|████████▎ | 247/296 [00:45<00:05,  8.51it/s]

...

 24%|██▍       | 502/2088 [04:12<10:45,  2.46it/s]

cant remove silence, path=coswara_wavs/train\n\C7Km0KttQRMMM6UoyocajfgZAOB3_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/train\n\C7Km0KttQRMMM6UoyocajfgZAOB3_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


 25%|██▍       | 512/2088 [04:16<12:56,  2.03it/s]

cant remove silence, path=coswara_wavs/train\n\cbmcTV6z0NVgO0i6yeYyAO7AoI72_cough-heavy.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....
cant remove silence, path=coswara_wavs/train\n\cbmcTV6z0NVgO0i6yeYyAO7AoI72_cough-shallow.wav, error -> AttributeError: 'int' object has no attribute 'samples'. Skipping....


 26%|██▋       | 549/2088 [04:34<13:37,  1.88it/s]

In [ ]:
print("Done!")
